In [ ]:
# =======================
# Regresión lineal simple
# =======================

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt

# doy x e y
x = np.array([25.2, 15.6, 26, 24, 39.2, 17.6, 3.6, 24, 10, 8.8, 35.2, 22.8, 31.6, 6, 11.2, 5.2, 22.4, 20.4, 31.2, 19.6])
y = np.array([10, 5, 9, 8, 10, 8, 1, 9, 3, 3, 10, 5, 10, 2, 4, 4, 9, 6, 10, 7])

# creo matriz de unos y x
X = np.array([np.ones(len(x)), np.array(x)]).transpose()

# calculo la pendiente y el término independiente
w = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)


def plot_all(w0,w1):

    plt.figure(figsize=(8, 4))
    ax = plt.axes()
    ax.scatter(x, y)
    ax.set_ylim([0,12])
    ax.set_xlim([0,45])
    ax.set_xlabel('Ingresos')
    ax.set_ylabel('Felicidad')
    e = np.linspace(0,45,len(x))
    z = w0+w1*e

    plt.plot(e, z, '-r', label='h(x)='+str(round(w0,2))+' + '+ str(round(w1,2))+'x')
    plt.legend(loc='upper left')
    plt.show()

plot_all(w[0],w[1])


# Evaluación del modelo
predictions = [ (w[0] + w[1] * x_i) for x_i in x]
len(predictions), len(y)

sqrt(mean_squared_error(y,predictions))

In [ ]:
# =======================
# Regresión lineal múltiple
# =======================


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler



sns.set_style('whitegrid')

# cargo datos
USA_Housing = pd.read_csv('data/USA_Housing.csv')

# quiero predecir la target prize, así que hago histograma del target, 
# si tiene forma de campana de gauss es ideal para aplicar regresión lineal
sns.histplot(USA_Housing['Price'], color='Blue', kde = True)
plt.show()


# Correlación con el target
corr = df.corr()["MEDV"].sort_values(ascending=False)
corr

# correlacion con el resto para evitar que estén correlacionadas
USA_Housing.corr(numeric_only= True)
sns.heatmap(USA_Housing.corr(numeric_only = True),annot=True);
plt.show();

# definimos features y target
# Feautures
X = USA_Housing[['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms',
       'Avg. Area Number of Bedrooms', 'Area Population']]

# Target
y = USA_Housing['Price']

# Separo train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# aquí podríamos escalar aquellas columnas que veamos que tienen 
# escalas muy distintas(valores pequeños, cientos, decimales...)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Ajustamos SOLO con train
X_train_scaled = scaler.fit_transform(X_train)
# Transformamos test con los parámetros de train
X_test_scaled = scaler.transform(X_test)
# volvemos a dataframe
X_train_scaled = pd.DataFrame(X_train_scaled,columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled,columns=X_test.columns,index=X_test.index)


# Creamos el modelo
lm = LinearRegression()

# Entrenamos con los datos de train
lm.fit(X_train, y_train)

# vemos el término independiente
lm.intercept_

# vemos los coeficientes
coef_df = pd.DataFrame(lm.coef_, X.columns, columns=['Coefficient'])
coef_df


# predecimos
predictions = lm.predict(X_test)
pred_train = lm.predict(X_train)


# por último evaluamos
# Train error
print('MAE train', metrics.mean_absolute_error(y_train, pred_train))
print('MSE train', metrics.mean_squared_error(y_train, pred_train))
print('RMSE train', np.sqrt(metrics.mean_squared_error(y_train, pred_train)))
print('R2 train', r2_score(y_train, pred_train))

# Test error
print('MAE test', metrics.mean_absolute_error(y_test, predictions))
print('MSE test', metrics.mean_squared_error(y_test, predictions))
print('RMSE test', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('R2 test', r2_score(y_test, predictions))

# dibujamos las predicciones
plt.figure(figsize=(6,6))
plt.scatter(y_test, predictions, alpha=0.7)
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()])
plt.xlabel("Valores reales (MEDV)")
plt.ylabel("Predicciones")
plt.title("Predicciones vs valores reales")
plt.show()


# para conocer ahora la importancia de las variables, tenemos que estandarizar
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)

lm_scaled = LinearRegression()
lm_scaled.fit(X_train_scaled, y_train)

feat_coef = pd.DataFrame(lm_scaled.coef_, X_train.columns, columns=['importance_standarized']).sort_values('importance_standarized',ascending=False)

# nos da un gráfico de las variables más o menos importantes
features = feat_coef.sort_values('importance_standarized')
plt.barh(features.index,features.importance_standarized)
plt.show()

# una vez que tenemos la importancia de las variables, eliminamos las menos importantes del modelo
X_train.drop(columns='Avg. Area Number of Bedrooms',inplace=True)
X_test.drop(columns='Avg. Area Number of Bedrooms',inplace=True)

lm2 = LinearRegression()
lm2.fit(X_train,y_train)

pred2 = lm2.predict(X_test)

# y comprobamos que se comete el mismo error, es decir el modelo es igual de bueno, pero mejorando rendimiento
print('MAE test', metrics.mean_absolute_error(y_test, pred2))
print('MSE test', metrics.mean_squared_error(y_test, pred2))
print('RMSE test', np.sqrt(metrics.mean_squared_error(y_test, pred2)))
print('R2 test', lm2.score(X_test,y_test))

In [ ]:
# =======================
# Regularización Regresión lineal (hacer el modelo más sencillo) (esto no lo he hecho, está solo en la práctica obligatoria del sprint 10)
# =======================

# si el error de train y de test es muy diferente, el modelo no generaliza bien, lo vemos
lr = LinearRegression()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
#baseline_error = metrics.mean_squared_error(y_test, y_pred)

print("Train MSE:", metrics.mean_squared_error(y_train, lr.predict(X_train)))
print("Test MSE:", metrics.mean_squared_error(y_test, lr.predict(X_test)))
# print("Test RMSE:", np.sqrt(metrics.mean_squared_error(y_test, lr.predict(X_test))))

